### Preprocessing the input data

In [3]:
import numpy as np
import tensorflow as tf

batch_size = 4
img_height = 299
img_width = 299

data_dir = '../data/train'

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)
print("Train OK")

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 56358 files belonging to 5 classes.
Using 50723 files for training.
Train OK
Found 56358 files belonging to 5 classes.
Using 5635 files for validation.


### InceptionV3


In [4]:
# Downloading the model
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(weights='imagenet')

### Customizing the model

In [ ]:
from keras import Model
from keras.layers import Dense
import keras


EPOCHS = 300
PATIENCE = 30

corn_output = Dense(5, activation='softmax')
corn_output = corn_output(model.layers[-2].output)
corn_input = model.input
    

corn_model = Model(inputs=corn_input, outputs=corn_output)

for layer in corn_model.layers[:-1]:
    layer.trainable

    
corn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='accuracy',
        patience=PATIENCE,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='checkpoint_inceptionv3_300.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    tf.keras.callbacks.CSVLogger(
        "Log", 
        separator=",", 
        append=False
    )
]

corn_model.fit(
    train_ds,
    steps_per_epoch=169,
    epochs=EPOCHS,
    verbose=2,
    validation_data=val_ds,
    callbacks=callbacks_list
)

model.save('inceptionv3_300.h5')

Epoch 1/300
